In [1]:
from langchain_community.utilities import SQLDatabase

In [2]:
db = SQLDatabase.from_uri("sqlite:///../meokten/meokten.db")

In [3]:
# Print the database dialect (sqlite)
print(f"db dialect : {db.dialect}")

# Print the list of usable table names in the database
print(f"table names in db : {db.get_usable_table_names()}")

# Execute an SQL query
db.run("SELECT * FROM restaurants LIMIT 5;")

db dialect : sqlite
table names in db : ['menus', 'restaurants']


"[(1, '이조갈비', '서울 중구 다산로39길 11-12 (무학동 58)', '37.5639175934235', '127.01485625085', '신당역 6호선(267m)', 'ZC5klhd08ME_0', 'https://www.youtube.com/watch?v=ZC5klhd08ME'), (2, '서령', '서울 중구 소월로 10 1층 (남대문로5가 120)', '37.5585411616418', '126.975411166003', '회현역 4호선(268m)', 'SvJ5mdneRSA_0', 'https://www.youtube.com/watch?v=SvJ5mdneRSA'), (3, '영등포 함흥냉면', '서울 영등포구 영등포로42길 6 (영등포동3가 7-32)', '37.5189887922679', '126.906774943721', '영등포역 1호선(374m)', '1Aj5KPmsxMU_0', 'https://www.youtube.com/watch?v=1Aj5KPmsxMU'), (4, '참조은,콩', '서울 동대문구 왕산로33길 16 1층 (제기동 635-12)', '37.5802186684531', '127.042361119212', '청량리역 1호선(210m)', 'inq2G9jzdQY_0', 'https://www.youtube.com/watch?v=inq2G9jzdQY'), (5, '감초식당', '서울 동대문구 약령서길 28 (제기동 892-12)', '37.5795666696962', '127.036256927579', '제기동역 1호선(192m)', 'inq2G9jzdQY_1', 'https://www.youtube.com/watch?v=inq2G9jzdQY')]"

In [4]:
from typing import Any

from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda, RunnableWithFallbacks
from langgraph.prebuilt import ToolNode


# Error handling function
def handle_tool_error(state) -> dict:
    # Check error information
    error = state.get("error")
    # Check tool information
    tool_calls = state["messages"][-1].tool_calls
    # Wrap with ToolMessage and return
    return {
        "messages": [
            ToolMessage(
                content=f"Here is the error: {repr(error)}\n\nPlease fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }


# Create a ToolNode to handle errors and surface them to the agent
def create_tool_node_with_fallback(tools: list) -> RunnableWithFallbacks[Any, dict]:
    """
    Create a ToolNode with a fallback to handle errors and surface them to the agent.
    """
    # Add fallback behavior for error handling to the ToolNode
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )

In [5]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI

# Create SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db, llm=ChatOpenAI(model="gpt-4o"))

# Get the list of available tools from the SQLDatabaseToolkit
tools = toolkit.get_tools()
for tool in tools:
    print(f"class_name : {tool.__class__}")
    print(f"description : {tool.description}")
    print()

class_name : <class 'langchain_community.tools.sql_database.tool.QuerySQLDatabaseTool'>
description : Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

class_name : <class 'langchain_community.tools.sql_database.tool.InfoSQLDatabaseTool'>
description : Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

class_name : <class 'langchain_community.tools.sql_database.tool.ListSQLDatabaseTool'>
description : Input is an empty string, output is a comma-separated list of tables in the database.

class_name : <class 'lang

In [6]:
# Select the tool for listing available tables in the database
list_tables_tool = next(tool for tool in tools if tool.name == "sql_db_list_tables")

# Select the tool for retrieving the DDL of a specific table
get_schema_tool = next(tool for tool in tools if tool.name == "sql_db_schema")

# Print the list of all tables in the database
print(f'list_tables :\n{list_tables_tool.invoke("")}')

print("-" * 100)

# Print the DDL information for the Artist table
print(f'schema :\n{get_schema_tool.invoke("menus")}')
print(f'schema :\n{get_schema_tool.invoke("restaurants")}')

list_tables :
menus, restaurants
----------------------------------------------------------------------------------------------------
schema :

CREATE TABLE menus (
	id INTEGER, 
	restaurant_id INTEGER, 
	menu_type TEXT, 
	menu_name TEXT NOT NULL, 
	menu_review TEXT, 
	PRIMARY KEY (id), 
	FOREIGN KEY(restaurant_id) REFERENCES restaurants (id)
)

/*
3 rows from menus table:
id	restaurant_id	menu_type	menu_name	menu_review
1	1	한식	냉동 삼겹살	냉동 삼겹살은 다른 양념과 함께 싸먹을 수 있는 것이 매력이다. 베이컨처럼 계속 먹을 수 있어 위험하다.
2	1	한식	생 삼겹살	생 삼겹살은 소금만 조금 찍어 먹으면 된다. 두껍기 때문에 어느 순간 멈추게 된다.
3	1	한식	게장 된장찌개	게장 된장찌개는 전설적이다. 달콤한 무와 게의 맛이 잘 어우러져 있으며, 집에서만 맛볼 수 있는 맛이다.
*/
schema :

CREATE TABLE restaurants (
	id INTEGER, 
	name TEXT NOT NULL, 
	address TEXT NOT NULL, 
	latitude TEXT, 
	longitude TEXT, 
	station_name TEXT, 
	video_id TEXT, 
	video_url TEXT, 
	PRIMARY KEY (id), 
	UNIQUE (video_id)
)

/*
3 rows from restaurants table:
id	name	address	latitude	longitude	station_name	video_id	video_url
1	이조갈비	서울 중구 다산로39길 11-12 (무학동 58

In [7]:
from langchain_core.tools import tool


# Query execution tool
@tool
def db_query_tool(query: str) -> str:
    """
    Run SQL queries against a database and return results
    Returns an error message if the query is incorrect
    If an error is returned, rewrite the query, check, and retry
    """
    # Execute query
    result = db.run_no_throw(query)

    # Error: Return error message if no result
    if not result:
        return "Error: Query failed. Please rewrite your query and try again."
    # Success: Return the query execution result
    return result

In [8]:
print(db_query_tool.invoke("SELECT * FROM restaurants LIMIT 10;"))

[(1, '이조갈비', '서울 중구 다산로39길 11-12 (무학동 58)', '37.5639175934235', '127.01485625085', '신당역 6호선(267m)', 'ZC5klhd08ME_0', 'https://www.youtube.com/watch?v=ZC5klhd08ME'), (2, '서령', '서울 중구 소월로 10 1층 (남대문로5가 120)', '37.5585411616418', '126.975411166003', '회현역 4호선(268m)', 'SvJ5mdneRSA_0', 'https://www.youtube.com/watch?v=SvJ5mdneRSA'), (3, '영등포 함흥냉면', '서울 영등포구 영등포로42길 6 (영등포동3가 7-32)', '37.5189887922679', '126.906774943721', '영등포역 1호선(374m)', '1Aj5KPmsxMU_0', 'https://www.youtube.com/watch?v=1Aj5KPmsxMU'), (4, '참조은,콩', '서울 동대문구 왕산로33길 16 1층 (제기동 635-12)', '37.5802186684531', '127.042361119212', '청량리역 1호선(210m)', 'inq2G9jzdQY_0', 'https://www.youtube.com/watch?v=inq2G9jzdQY'), (5, '감초식당', '서울 동대문구 약령서길 28 (제기동 892-12)', '37.5795666696962', '127.036256927579', '제기동역 1호선(192m)', 'inq2G9jzdQY_1', 'https://www.youtube.com/watch?v=inq2G9jzdQY'), (6, '논현역 하이콴', '서울 서초구 신반포로 340 1층 (반포동 706-8)', '37.5107449522232', '127.020890081267', '논현역 신분당선(79m)', 'pY1kORvBEYs_0', 'https://www.youtube.com/watch?v=p

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Define a system message to check SQL queries for common mistakes
query_check_system = """You are a SQL expert with a strong attention to detail.
Double check the SQLite query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

You will call the appropriate tool to execute the query after running this check."""

# Create the prompt
query_check_prompt = ChatPromptTemplate.from_messages(
    [("system", query_check_system), ("placeholder", "{messages}")]
)

# Create the Query Checker chain
query_check = query_check_prompt | ChatOpenAI(model="gpt-4o", temperature=0).bind_tools(
    [db_query_tool], tool_choice="db_query_tool"
)

In [10]:
# Execute the query check node using the user's message
response = query_check.invoke(
    {"messages": [("user", "SELECT * FROM restaurants LIMITS 5;")]}
)
print(response.tool_calls[0])

{'name': 'db_query_tool', 'args': {'query': 'SELECT * FROM restaurants LIMIT 5;'}, 'id': 'call_pzPRoQGAe3Jwim7aU3IVoUER', 'type': 'tool_call'}


In [11]:
from typing import Annotated, Literal

from langchain_core.messages import AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from typing import List

from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.output_parsers import JsonOutputParser


# Define the agent's state
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


# Create a new graph
workflow = StateGraph(State)


# Add a node for the first tool call
def first_tool_call(state: State) -> dict[str, list[AIMessage]]:
    return {
        "messages": [
            AIMessage(
                content="",
                tool_calls=[
                    {
                        "name": "sql_db_list_tables",
                        "args": {},
                        "id": "initial_tool_call_abc123",
                    }
                ],
            )
        ]
    }


# Define a function to check query accuracy with a model
def model_check_query(state: State) -> dict[str, list[AIMessage]]:
    """
    Use this tool to check that your query is correct before you run it
    """
    return {"messages": [query_check.invoke({"messages": [state["messages"][-1]]})]}


# Add a node for the first tool call
workflow.add_node("first_tool_call", first_tool_call)

# Add nodes for the first two tools
workflow.add_node(
    "list_tables_tool", create_tool_node_with_fallback([list_tables_tool])
)
workflow.add_node("get_schema_tool", create_tool_node_with_fallback([get_schema_tool]))

# Add a model node to select relevant tables based on the question and available tables
model_get_schema = ChatOpenAI(model="gpt-4o", temperature=0).bind_tools(
    [get_schema_tool]
)
workflow.add_node(
    "model_get_schema",
    lambda state: {
        "messages": [model_get_schema.invoke(state["messages"])],
    },
)


# 최종 상태를 나타내는 도구 설명
class SubmitFinalAnswer(BaseModel):
    """쿼리 결과를 기반으로 사용자에게 최종 답변 제출"""

    final_answer: str = Field(..., description="The final answer to the user")


# 쿼리 생성을 위한 프롬프트
QUERY_GEN_INSTRUCTION = """You are a SQL expert with a strong attention to detail.

You can define SQL queries to retrieve information from a database.

Read the messages below and identify the user question, table schemas, and any previous query results or errors.

IMPORTANT: Only use tables and columns that are explicitly mentioned in the schema information provided. Do NOT assume the existence of any tables or columns that are not explicitly shown in the schema.

The database only has two tables: 'restaurants' and 'menus'. Do not try to query any other tables.

Your task is to:

1. If there's not any query result that makes sense to answer the question, create a syntactically correct SQLite query to answer the user question. DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

2. When creating queries, ALWAYS use SELECT * or explicitly list ALL columns from the tables you're querying to ensure all available data is returned. This is especially important for location data like coordinates.

3. If you create a query, response ONLY with the query statement. For example, "SELECT * FROM restaurants;" or "SELECT id, name, address, latitude, longitude, rating FROM restaurants;"

4. If a query was already executed, but there was an error, respond with the same error message you found. For example: "Error: Pets table doesn't exist"

5. If a query was already executed successfully, DO NOT create a new query. Instead, respond with "QUERY_EXECUTED_SUCCESSFULLY" so the system knows to proceed to the answer generation step.

6. If you encounter any issues that prevent you from creating a valid query, respond with "Error: [explanation of the issue]"
"""

query_gen_prompt = ChatPromptTemplate.from_messages(
    [("system", QUERY_GEN_INSTRUCTION), ("placeholder", "{messages}")]
)


query_gen = query_gen_prompt | ChatOpenAI(model="gpt-4o", temperature=0)


# 답변 생성을 위한 프롬프트
ANSWER_GEN_INSTRUCTION = """당신은 SQL 쿼리 결과를 해석하여 사용자에게 친절하고 명확한 답변을 제공하는 전문가입니다.
제공되는 정보들은 성시경의 유튜브 영상 중 "먹을텐데"에 대한 정보들 입니다.

주어진 쿼리 결과를 분석하고, 사용자의 질문에 직접적으로 답변해주세요.

답변 작성 시 다음 사항을 지켜주세요:
1. 맛집 정보를 제공할 때는 이름, 주소, 지하철역을 제공 해주세요.
2. 삭당의 메뉴들과 후기를 충분하게 제공 해주세요.
3. 정보가 부족한 경우, 찾을 수 없다는 메시지를 제공 해주세요.
4. 사용자가 이해하기 쉬운 자연스러운 한국어로 답변하세요.

OUTPUT_FORMAT:
{{
    "answer": "답변 내용"
    "info":{{
        "name": "식당 이름",
        "address": "식당 주소",
        "subway": "식당 지하철역",
        "lat": "식당 위도",
        "lng": "식당 경도",
        "menu": "식당 메뉴",
        "review": "식당 후기"
    }}
}}
"""


class Info(BaseModel):
    name: str = Field(..., description="식당 이름")
    address: str = Field(..., description="식당 주소")
    subway: str = Field(..., description="식당 지하철역")
    lat: str = Field(..., description="식당 위도")
    lng: str = Field(..., description="식당 경도")
    menu: str = Field(..., description="식당 메뉴")
    review: str = Field(..., description="식당 후기")


class Answers(BaseModel):
    answer: str = Field(..., description="답변 내용")
    info: List[Info] = Field(..., description="식당 정보")


answer_gen_prompt = ChatPromptTemplate.from_messages(
    [("system", ANSWER_GEN_INSTRUCTION), ("placeholder", "{messages}")]
)

answer_gen = (
    answer_gen_prompt
    | ChatOpenAI(model="gpt-4o", temperature=0)
    | JsonOutputParser(pydantic_object=Answers)
)


# Define conditional edges
def should_continue(
    state: State,
) -> Literal[END, "correct_query", "query_gen", "generate_answer"]:
    last_message = state["messages"][-1]

    # 메시지 내용이 있는 경우
    if hasattr(last_message, "content") and isinstance(last_message.content, str):
        # 1) Terminate if the message starts with "Answer:"
        if last_message.content.startswith("Answer:"):
            return END
        # 2) 쿼리가 성공적으로 실행되었으면 답변 생성 노드로 이동
        elif last_message.content == "QUERY_EXECUTED_SUCCESSFULLY":
            return "generate_answer"
        # 3) 오류가 있으면 쿼리 생성 노드로 돌아감
        elif last_message.content.startswith("Error:"):
            return "query_gen"
        # 4) 일반 텍스트 응답이 있으면 (영어로 된 답변 등) 종료
        elif len(last_message.content) > 20 and not last_message.content.startswith(
            "SELECT"
        ):
            return END

    # 5) 반복 횟수 제한을 위한 안전장치
    if len(state["messages"]) > 20:
        return END

    # 기본적으로 쿼리 검증 노드로 이동
    return "correct_query"


# Define the query generation node
def query_gen_node(state: State):
    try:
        # 이전 메시지에 이미 쿼리 결과가 있는지 확인
        for message in reversed(state["messages"][:-1]):  # 마지막 메시지 제외
            if (
                hasattr(message, "name")
                and message.name == "db_query_tool"
                and hasattr(message, "content")
                and not message.content.startswith("Error:")
            ):
                # 쿼리 결과가 있으면 QUERY_EXECUTED_SUCCESSFULLY 반환
                return {"messages": [AIMessage(content="QUERY_EXECUTED_SUCCESSFULLY")]}

        # 쿼리 생성
        message = query_gen.invoke(state)

        # 이미 답변 형식이면 그대로 반환
        if (
            hasattr(message, "content")
            and isinstance(message.content, str)
            and len(message.content) > 50  # 긴 텍스트는 답변으로 간주
            and not message.content.startswith("SELECT")
            and not message.content.startswith("Error:")
        ):
            # 답변이 "Answer:"로 시작하지 않으면 추가
            if not message.content.startswith("Answer:"):
                message.content = f"Answer: {message.content}"
            return {"messages": [message]}

        # 일반적인 쿼리 또는 오류 메시지
        return {"messages": [message]}

    except Exception as e:
        return {
            "messages": [
                AIMessage(content=f"Error: 쿼리 생성 중 오류가 발생했습니다: {str(e)}")
            ]
        }


# 쿼리 실행 결과를 처리하는 노드
def process_query_result(state: State):
    last_message = state["messages"][-1]

    # 쿼리 실행 결과가 있으면 성공 신호 반환
    if (
        hasattr(last_message, "name")
        and last_message.name == "db_query_tool"
        and hasattr(last_message, "content")
        and not last_message.content.startswith("Error:")
    ):
        return {"messages": [AIMessage(content="QUERY_EXECUTED_SUCCESSFULLY")]}

    # 결과가 없거나 오류인 경우 그대로 반환
    return {"messages": [last_message]}


# 답변 생성 노드 정의
def generate_answer_node(state: State):
    try:
        # 쿼리 결과 찾기
        query_result = None
        for message in reversed(state["messages"]):
            if (
                hasattr(message, "name")
                and message.name == "db_query_tool"
                and hasattr(message, "content")
                and not message.content.startswith("Error:")
            ):
                query_result = message.content
                break

        if not query_result:
            return {
                "messages": [
                    AIMessage(
                        content="Answer: 죄송합니다, 쿼리 결과를 찾을 수 없습니다."
                    )
                ]
            }

        # 사용자 질문 찾기
        user_question = None
        for message in state["messages"]:
            if hasattr(message, "type") and message.type == "human":
                user_question = message.content
                break

        # 답변 생성을 위한 컨텍스트 구성
        try:
            # JSON 파싱 오류 방지를 위한 수정
            import json
            from pydantic import ValidationError

            # 답변 생성 시도
            answer_context = {
                "messages": [
                    {
                        "role": "user",
                        "content": f"질문: {user_question}\n\n쿼리 결과: {query_result}",
                    }
                ]
            }

            # 직접 LLM 호출 후 결과 처리
            llm_response1 = ChatOpenAI(model="gpt-4o", temperature=0).invoke(
                answer_gen_prompt.format_messages(messages=answer_context["messages"])
            )
            print(f"llm_response1: {llm_response1}")
            llm_response2 = answer_gen.invoke({"messages":answer_context["messages"]})
            print(f"llm_response2: {llm_response2}")
            
            # 응답에서 JSON 부분 추출 시도
            content_str = llm_response1.content
            
            # JSON 형식 검증 및 파싱
            try:
                # JSON 문자열 추출 시도 (Answers: {...} 형식에서 {...} 부분만 추출)
                if "Answers:" in content_str:
                    json_str = content_str.split("Answers:", 1)[1].strip()
                else:
                    json_str = content_str

                # JSON 파싱
                answer_dict = json.loads(json_str)

                # Pydantic 모델로 변환
                if "info" in answer_dict:
                    # 이미 올바른 형식
                    answer_obj = answer_dict
                elif "Answers" in answer_dict and "info" in answer_dict["Answers"]:
                    # Answers 키가 있는 경우
                    answer_obj = answer_dict["Answers"]
                else:
                    # 형식이 맞지 않는 경우
                    raise ValueError("응답 형식이 올바르지 않습니다")

                content = f"Answer: {answer_dict}"

            except (json.JSONDecodeError, ValidationError, ValueError, KeyError) as e:
                # JSON 파싱 실패 시 원본 텍스트 사용
                content = f"Answer: {content_str}"

        except Exception as e:
            # LLM 호출 실패 시 기본 응답
            content = f"Answer: 죄송합니다, 쿼리 결과를 해석하는 중 오류가 발생했습니다: {str(e)}"

        return {"messages": [AIMessage(content=content)]}

    except Exception as e:
        return {
            "messages": [
                AIMessage(
                    content=f"Answer: 죄송합니다, 답변 생성 중 오류가 발생했습니다: {str(e)}"
                )
            ]
        }


# Add the nodes
workflow.add_node("query_gen", query_gen_node)
workflow.add_node("correct_query", model_check_query)
workflow.add_node("execute_query", create_tool_node_with_fallback([db_query_tool]))
workflow.add_node("process_query_result", process_query_result)
workflow.add_node("generate_answer", generate_answer_node)

# Specify edges between nodes
workflow.add_edge(START, "first_tool_call")
workflow.add_edge("first_tool_call", "list_tables_tool")
workflow.add_edge("list_tables_tool", "model_get_schema")
workflow.add_edge("model_get_schema", "get_schema_tool")
workflow.add_edge("get_schema_tool", "query_gen")
workflow.add_conditional_edges(
    "query_gen",
    should_continue,
)
workflow.add_edge("correct_query", "execute_query")
workflow.add_edge("execute_query", "process_query_result")
workflow.add_edge("process_query_result", "query_gen")
workflow.add_edge("generate_answer", END)

# Compile the workflow into an executable app
app = workflow.compile(checkpointer=MemorySaver())

In [12]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableConfig


from langgraph_utils import invoke_graph, random_uuid

In [14]:
app.invoke(
    {
        "messages": '서울에서 한식 맛집 여러곳 추천해줘'
    },
    RunnableConfig(recursion_limit=30, configurable={"thread_id": random_uuid()})
)


llm_response1: content='{\n    "answer": "서울에서 추천할 만한 한식 맛집을 몇 군데 소개해드릴게요.",\n    "info": [\n        {\n            "name": "이조갈비",\n            "address": "서울 중구 다산로39길 11-12 (무학동 58)",\n            "subway": "신당역 6호선(267m)",\n            "lat": "37.5639175934235",\n            "lng": "127.01485625085",\n            "menu": "갈비",\n            "review": "이조갈비는 부드럽고 맛있는 갈비로 유명합니다. 고기의 질이 좋고 양념이 잘 배어 있어 많은 사람들이 찾는 곳입니다."\n        },\n        {\n            "name": "서령",\n            "address": "서울 중구 소월로 10 1층 (남대문로5가 120)",\n            "subway": "회현역 4호선(268m)",\n            "lat": "37.5585411616418",\n            "lng": "126.975411166003",\n            "menu": "한식",\n            "review": "서령은 전통 한식을 현대적으로 재해석한 메뉴로 인기를 끌고 있습니다. 깔끔한 인테리어와 정갈한 음식이 특징입니다."\n        },\n        {\n            "name": "영등포 함흥냉면",\n            "address": "서울 영등포구 영등포로42길 6 (영등포동3가 7-32)",\n            "subway": "영등포역 1호선(374m)",\n            "lat": "37.5189887922679",\n            "lng": "126.906774943721",

{'messages': [HumanMessage(content='서울에서 한식 맛집 여러곳 추천해줘', additional_kwargs={}, response_metadata={}, id='a3015f5f-6b27-4587-a1aa-92d5562f3ea8'),
  AIMessage(content='', additional_kwargs={}, response_metadata={}, id='38462053-1393-42fa-af0f-f8225f3d5f40', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'initial_tool_call_abc123', 'type': 'tool_call'}]),
  ToolMessage(content='menus, restaurants', name='sql_db_list_tables', id='61aed26e-8d34-44e5-af8e-a6ec26184495', tool_call_id='initial_tool_call_abc123'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PlL6IHIl263ThjnmYp5je7jg', 'function': {'arguments': '{"table_names": "restaurants"}', 'name': 'sql_db_schema'}, 'type': 'function'}, {'id': 'call_GkFqqkCYhhmHbPjNI7JPg1PR', 'function': {'arguments': '{"table_names": "menus"}', 'name': 'sql_db_schema'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 154, 'total_tokens': 203, 'compl

In [ ]:
output = invoke_graph(
    app,
    {
        "messages": '서울에서 한식 맛집 여러곳 추천해줘'
    },
    RunnableConfig(recursion_limit=30, configurable={"thread_id": random_uuid()}),
    return_result=True,
)

In [ ]:
print(output["final_result"]["messages"][0].content)

In [ ]:
import requests

url = "https://github.com/jinucho/Meokten/raw/main/meokten/meokten.db"
file_path = "meokten.db"

try:
    response = requests.get(url, stream=True)
    response.raise_for_status()  # HTTP 오류가 발생하면 예외 발생

    with open(file_path, "wb") as file:
        for chunk in response.iter_content(1024):
            file.write(chunk)

    print(f"파일이 {file_path}로 성공적으로 다운로드되었습니다.")

except requests.exceptions.RequestException as e:
    print("파일 다운로드 중 오류 발생:", e)


db = SQLDatabase.from_uri(f"sqlite:///{file_path}")

In [ ]:
db.run("SELECT * FROM menus LIMIT 5;")